In [1]:
from pathlib import Path

import autochem as ac
import numpy as np
import polars as pl

DATA_PATH = Path("data")
T_vals = [
    300.0,
    400.0,
    500.0,
    625.0,
    750.0,
    875.0,
    1000.0,
    1125.0,
    1250.0,
    1375.0,
    1500.0,
    1750.0,
    2000.0,
    2250.0,
    2500.0,
]
T_range = (T_vals[0], T_vals[-1])
s1_file = DATA_PATH / "kane" / "W1" / "pf.dat"
s2_file = DATA_PATH / "kane" / "W2a" / "pf.dat"

r12_str = r"""
W1->W2a

P\T           300       400       500       625       750       875     1e+03  1.12e+03  1.25e+03  1.37e+03   1.5e+03  1.75e+03     2e+03  2.25e+03   2.5e+03
0.1      7.01e-09  0.000325     0.144      15.9       321  2.38e+03  9.35e+03   2.4e+04  4.69e+04   7.7e+04  1.13e+05       ***       ***       ***       ***
0.3      7.32e-09  0.000367     0.185      24.4       563  4.62e+03  1.96e+04  5.35e+04  1.09e+05  1.85e+05  2.79e+05       ***       ***       ***       ***
1        7.53e-09  0.000405     0.234      36.9       985  9.04e+03  4.19e+04  1.23e+05  2.66e+05   4.7e+05  7.26e+05       ***       ***       ***       ***
3        7.62e-09  0.000429     0.274      50.1  1.52e+03  1.55e+04  7.86e+04  2.48e+05  5.67e+05  1.05e+06  1.67e+06  3.26e+06       ***       ***       ***
10       7.66e-09  0.000443     0.306        64  2.22e+03  2.54e+04  1.42e+05  4.88e+05   1.2e+06  2.33e+06  3.86e+06  7.92e+06       ***       ***       ***
30       7.67e-09  0.000449     0.323      73.9  2.83e+03  3.59e+04   2.2e+05  8.19e+05  2.15e+06  4.41e+06  7.61e+06  1.64e+07  2.72e+07       ***       ***
100      7.68e-09  0.000452     0.332      80.7  3.37e+03  4.69e+04  3.16e+05  1.28e+06  3.63e+06  7.93e+06  1.44e+07   3.3e+07  5.66e+07       ***       ***
O-O      2.71e-07  0.000652     0.344      86.3  3.99e+03  6.54e+04  5.48e+05  2.92e+06  1.12e+07  3.42e+07  8.67e+07  3.79e+08  1.16e+09  2.77e+09  5.59e+09
"""
r21_str = r"""
W2a->W1

P\T           300       400       500       625       750       875     1e+03  1.12e+03  1.25e+03  1.37e+03   1.5e+03  1.75e+03     2e+03  2.25e+03   2.5e+03
0.1      2.28e-08   0.00108     0.491      55.8  1.15e+03  8.66e+03  3.44e+04  8.91e+04  1.75e+05   2.9e+05  4.28e+05       ***       ***       ***       ***
0.3      2.38e-08   0.00122     0.634      85.6  2.01e+03  1.68e+04   7.2e+04  1.98e+05  4.09e+05  6.98e+05  1.05e+06       ***       ***       ***       ***
1        2.45e-08   0.00135       0.8       129  3.52e+03  3.29e+04  1.54e+05  4.57e+05  9.94e+05  1.77e+06  2.75e+06       ***       ***       ***       ***
3        2.48e-08   0.00143     0.936       176  5.45e+03  5.65e+04  2.89e+05   9.2e+05  2.12e+06  3.94e+06  6.32e+06  1.25e+07       ***       ***       ***
10       2.49e-08   0.00148      1.05       225  7.93e+03  9.24e+04  5.23e+05  1.81e+06  4.48e+06  8.77e+06  1.46e+07  3.04e+07       ***       ***       ***
30        2.5e-08    0.0015       1.1       259  1.01e+04   1.3e+05  8.09e+05  3.04e+06  8.04e+06  1.66e+07  2.89e+07  6.29e+07  1.06e+08       ***       ***
100       2.5e-08   0.00151      1.14       283  1.21e+04   1.7e+05  1.16e+06  4.76e+06  1.36e+07  2.99e+07  5.45e+07  1.26e+08   2.2e+08       ***       ***
O-O      8.81e-07   0.00217      1.18       303  1.43e+04  2.38e+05  2.02e+06  1.08e+07   4.2e+07  1.29e+08  3.28e+08  1.45e+09  4.44e+09  1.07e+10  2.17e+10
"""

In [2]:
# Read in reactant data
s1_therm = ac.therm.data.from_messpf_output_string(s1_file.read_text(), "NH2NO", Hf=0)

# Read in product data
s2_therm = ac.therm.data.from_messpf_output_string(s2_file.read_text(), "NHNOH", Hf=0)

In [3]:
# Read in forward rate data
r12_rate = ac.rate.data.from_mess_channel_output(r12_str, order=1)

# Read in reverse rate data
r21_rate = ac.rate.data.from_mess_channel_output(r21_str, order=1)

In [4]:
# Divide forward rate by reverse rate
rate_ratio = r12_rate(T=T_vals, P=np.inf) / r21_rate(T=T_vals, P=np.inf)
rate_ratio

array([0.30760499, 0.30046083, 0.29152542, 0.28481848, 0.27902098,
       0.27478992, 0.27128713, 0.27037037, 0.26666667, 0.26511628,
       0.26432927, 0.26137931, 0.26126126, 0.2588785 , 0.25760369])

In [5]:
# Divide product partition function by reactant partition function
therm_ratio = s2_therm(T=T_vals) / s1_therm(T=T_vals)
therm_ratio

array([0.38239545, 0.35341934, 0.33343745, 0.31647849, 0.3048608 ,
       0.2965617 , 0.29034076, 0.28556089, 0.28178807, 0.27867752,
       0.27615306, 0.27215052, 0.26917326, 0.2668683 , 0.26503325])

In [6]:
percent_difference = 100 * (rate_ratio - therm_ratio) / ((rate_ratio + therm_ratio) / 2)
percent_difference

array([-21.67837863, -16.19823062, -13.41264389, -10.53057313,
        -8.8510455 ,  -7.62114955,  -6.78514541,  -5.4648911 ,
        -5.51418615,  -4.98764168,  -4.37527429,  -4.03771434,
        -2.98321586,  -3.03940811,  -2.8431058 ])

In [7]:
data = {
    "T": T_vals,
    r"% difference": percent_difference
}

df = pl.DataFrame(data)
df.write_csv("percent_difference.csv")

display(df)

T,% difference
f64,f64
300.0,-21.678379
400.0,-16.198231
500.0,-13.412644
625.0,-10.530573
750.0,-8.851046
…,…
1500.0,-4.375274
1750.0,-4.037714
2000.0,-2.983216
